# 畳み込みニューラルネットワークの学習

In [1]:
# coding: UTF-8

import chainer
from chainer import Variable, Chain, optimizers, serializers, datasets
import chainer.links as L
import chainer.functions as F

from chainer.datasets import tuple_dataset
from chainer import training, iterators
from chainer.training import extensions

import numpy as np

In [2]:
# MNISTデータの読み込み
mnist_data = datasets.get_mnist(ndim=3) # 1x28x28
train_data = mnist_data[0]
test_data = mnist_data[1]

#print("Train:", len(train_data))
#print("Test:", len(test_data))

Train: 60000
Test: 10000


In [3]:
# MNIST画像の表示
#import matplotlib
#%matplotlib inline
#import matplotlib.pyplot as plt

#index = 2
#plt.imshow(train_data[index][0].reshape(28, 28), cmap = 'gray')
#plt.title(train_data[index][1])
#plt.show()

In [9]:
# Chain の記述
class MyMNIST(Chain):
    def __init__(self):
        super(MyMNIST, self).__init__(
            # L.convolution2D(チャンネル数, フィルタ数, フィルタのサイズ)
            cnn1 = L.Convolution2D( 1, 15, 5), # 画像サイズ(1, 28, 28) → (15, 24, 24)
            cnn2 = L.Convolution2D(15, 40, 5), # 画像サイズ(15, 12, 12) → (40, 8, 8)
            l1 = L.Linear(640, 400), # 入力は 40 x 4 x 4 = 640 → 400
            l2 = L.Linear(400, 10), # 400 → 10
        )
    
    def __call__(self, x, t):
        return F.softmax_cross_entropy(self.predict(x), t) # 交差エントロピー
    
    def predict(self, x):
        # F.max_pooling_2d(入力画像, 領域のサイズ)
        h1 = F.max_pooling_2d(F.relu(self.cnn1(x)),  2) # 画像サイズ(15, 24, 24) → (15, 12, 12)
        h2 = F.max_pooling_2d(F.relu(self.cnn2(h1)), 2) # 画像サイズ(40,  8,  8) → (40,  4,  4)
        h3 = F.dropout(F.relu(self.l1(h2)))
        return self.l2(h3)

In [10]:
# モデルとoptimizerの設定
model = MyMNIST()
optimizer = optimizers.Adam() # Adam
optimizer.setup(model)

In [11]:
# 学習
iterator = iterators.SerialIterator(train_data, 500)
updater = training.StandardUpdater(iterator, optimizer)
trainer = training.Trainer(updater, (20, 'epoch'))
trainer.extend(extensions.ProgressBar())
trainer.run()

     total [##................................................]  4.17%
this epoch [#########################################.........] 83.33%
       100 iter, 0 epoch / 20 epochs
       inf iters/sec. Estimated time to finish: 0:00:00.
     total [####..............................................]  8.33%
this epoch [#################################.................] 66.67%
       200 iter, 1 epoch / 20 epochs
    1.2243 iters/sec. Estimated time to finish: 0:29:56.959996.
     total [######............................................] 12.50%
this epoch [#########################.........................] 50.00%
       300 iter, 2 epoch / 20 epochs
    1.2728 iters/sec. Estimated time to finish: 0:27:29.959497.
     total [########..........................................] 16.67%
this epoch [################..................................] 33.33%
       400 iter, 3 epoch / 20 epochs
    1.3062 iters/sec. Estimated time to finish: 0:25:31.139998.
     total [##########.............

In [12]:
# モデルの保存
serializers.save_npz('my_mnist.npz', model)